In [53]:
import json
import os

In [54]:
def create_table(db, start, end, id):
  table_name = db['table_names_original'][id]
  table_description = db['table_names'][id]
  all_columns = db['column_names_original']
  all_columns_description = db['column_names']
  database_name = db['db_id']
  table_columns = list(map(lambda column: column[1], all_columns[start:end+1]))
  table_column_descriptions = list(map(lambda column: column[1], all_columns_description[start:end+1]))

  table_column_types = db['column_types'][start:end+1]
  # print(db['primary_keys'][id], db['db_id'], id)
  primary_key = all_columns[int(db['primary_keys'][id])][1] if len(db['primary_keys']) > id else None
  return Table(id, database_name, table_name, table_description, table_columns, \
               table_column_descriptions, table_column_types, primary_key)


def build_database(db):
  columns = db['column_names_original']
  tables = []
  id,start = 0,1
  for index, column in enumerate(columns[1:]):
    #accumlate all the columns related to a particular table
    if column[0] != id:
      table = create_table(db, start, index, id)
      tables.append(table)
      start = index+1
      id += 1

  tables.append(create_table(db, start, len(columns)-1, id))
  return Database(db['db_id'], tables, columns, db['foreign_keys'])

In [55]:
class Database:
  def __init__(self, name, tables, columns, foreign_keys):
    self.name = name
    self.tables = tables
    self.columns = columns
    self.foreign_keys = foreign_keys

  def __str__(self):
    """Returns a string representation of the Database object."""
    return f"Database(name='{self.name}', tables={self.tables}), columns={self.columns}), foreign_keys={self.foreign_keys})"


class Table:
  def __init__(self, id, database_name ,name, description, columns, columns_description,column_types, primary_key):
    self.id = id
    self.database_name = database_name
    self.name = name
    self.description = description
    self.columns = columns
    self.columns_description = columns_description
    self.column_types = column_types
    self.primary_key = primary_key

  def __str__(self):
    """Returns a string representation of the Table object."""
    return f"Table(id={self.id}, name='{self.name}', description='{self.description}', columns={self.columns}, columns_description={self.columns_description}, column_types={self.column_types}, primary_key='{self.primary_key}')"


In [56]:
len(databases)

166

In [57]:
from elasticsearch import Elasticsearch

es = Elasticsearch('https://localhost:9200', http_auth=('elastic', 'ZXJxoo-rx_YWk1kbMs*b'), verify_certs=False)

C:\Users\Patron\miniconda3\Lib\site-packages\elasticsearch\_sync\client\__init__.py:399: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
C:\Users\Patron\AppData\Local\Temp\ipykernel_22180\1456957216.py:3: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch('https://localhost:9200', http_auth=('elastic', 'ZXJxoo-rx_YWk1kbMs*b'), verify_certs=False)


In [58]:
# Create an index (use a different index name if needed)
index_name = "database_catalogue_index"
es.indices.create(index=index_name)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<elastic_transport._node._urllib3_chain_certs.HTTPSConnection object at 0x000002378DF6A4E0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it))

In [59]:
class DatabaseCatalogueEncoder(json.JSONEncoder):
  def default(self, obj):
    if isinstance(obj, Table):
      # Override default behavior for Book objects
      return {
        "name" : obj.name,
        "description" : obj.description,
        # "columns" : obj.columns,
        "columns_description" : obj.columns_description,
        "db_name": obj.database_name,
        # "column_types" : obj.column_types,
        # "primary_key" : obj.primary_key
      }
    return super().default(obj)

In [60]:
tables_file_path = 'Documents/spider/tables.json'
index_name= "database_index"
with open(tables_file_path) as tables_file:
  db_list = json.load(tables_file)
  databases = []
  with open("catalogue.txt", "w") as file:
      for ind,db in enumerate(db_list):
          database = build_database(db)
          # Index the document
          for index, table in enumerate(database.tables):
              if index == 0:
                  file.write(json.dumps(table, cls=DatabaseCatalogueEncoder))
                  file.write("\n")
                  # print(json.dumps(table, cls=DatabaseCatalogueEncoder))
              # es.index(index=index_name, id=database.name +'_'+ str(index), body=json.dumps(table, cls=DatabaseCatalogueEncoder))
          databases.append(database)

print(databases[0])

Database(name='perpetrator', tables=[<__main__.Table object at 0x000002378D14FB30>, <__main__.Table object at 0x000002378D2BFBF0>]), columns=[[-1, '*'], [0, 'Perpetrator_ID'], [0, 'People_ID'], [0, 'Date'], [0, 'Year'], [0, 'Location'], [0, 'Country'], [0, 'Killed'], [0, 'Injured'], [1, 'People_ID'], [1, 'Name'], [1, 'Height'], [1, 'Weight'], [1, 'Home Town']]), foreign_keys=[[2, 9]])


In [51]:
query_sent = "what is the longest river that runs through a state that borders tennessee"
search_results = es.search(index="database_index", body={"query": {"match": {"columns_description": query_sent}}})

print(search_results)

{'took': 14, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 52, 'relation': 'eq'}, 'max_score': 7.504965, 'hits': [{'_index': 'database_index', '_id': 'geo6', '_score': 7.504965, '_source': {'name': 'river', 'description': 'river', 'columns_description': ['river name', 'length', 'country name', 'traverse']}}, {'_index': 'database_index', '_id': 'gas_company2', '_score': 6.8146, '_source': {'name': 'station_company', 'description': 'station company', 'columns_description': ['station id', 'company id', 'rank of the year']}}, {'_index': 'database_index', '_id': 'yelp0', '_score': 6.3409114, '_source': {'name': 'business', 'description': 'business', 'columns_description': ['bid', 'business id', 'name', 'full address', 'city', 'latitude', 'longitude', 'review count', 'is open', 'rating', 'state']}}, {'_index': 'database_index', '_id': 'world_13', '_score': 5.908356, '_source': {'name': 'countrylanguage', 'description': 'c

C:\Users\Patron\AppData\Roaming\Python\Python312\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [52]:
for hit in search_results['hits']['hits']:
    print(f"Title: {hit['_id']}")
    print("-" * 20)

Title: geo6
--------------------
Title: gas_company2
--------------------
Title: yelp0
--------------------
Title: world_13
--------------------
Title: school_player2
--------------------
Title: epinions_11
--------------------
Title: wedding0
--------------------
Title: party_host2
--------------------
Title: employee_hire_evaluation2
--------------------
Title: concert_singer1
--------------------


In [36]:
response = es.count(index="database_catalogue_index")

C:\Users\Patron\AppData\Roaming\Python\Python312\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [37]:
response

ObjectApiResponse({'count': 876, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})